# New Structural Reform Interface
This notebook is meant to demonstrate an under-construction interface for structural reforms. This notebook will be refactored or removed before being placed in production.

## Getting Started
To test out the new structural reform implementation:
1. Pip install this package in editable mode in a fresh Python environment.
1. In a separate window in VSCode or another editor, clone [this PR](https://github.com/PolicyEngine/policyengine-core/pull/329) from Anthony Volk's fork of `policyengine-core` and pip install it in editable mode into the same environment (note - this is critical to properly running).

## Introducing StructuralReform
The above changes to `-core` introduce a new class, `StructuralReform`, meant to improve upon how PolicyEngine develops new structural reforms. This class should remove some of the bloat & repetitiveness of structural reform development and add new features, while maintaining a familiar enough interface to allow easy porting of our existing codebase.

Let's imagine we're creating a sample reform. This reform will be simple: we will vastly reduce the EITC by altering the EITC variable to provide $4 to every tax unit that qualifies for it under current government rules. To get started, we will create a new file in the `reforms` folder, like with the old structural reform interface. Unlike with the old interface, we can separate variables into distinct files, as with parametric reforms.

The below shows a sample file containing our new EITC variable. Note: do not run the cell, as it will not function properly.

In [1]:
# policyengine_us/reforms/example_variables/eitc.py

from policyengine_us.model_api import *

class eitc(Variable):

    def formula(tax_unit, period, parameters):
      return 4


/opt/miniconda3/envs/struct-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


eitc_reform.py


KeyboardInterrupt: 

This is accompanied by a separate file that does the following:
* Instantiates a `StructuralReform` instance
* Passes the initializer a string representing the parameter we'll use to trigger this reform
* Indicates what we want to do with the variables that make up the reform

The new structural reform interface allows three operations that are the same as our previous implementation:
* Adding a variable: creating a new variable from scratch and adding it to the tax-benefit system
* Updating a variable: taking an existing variable and replacing part of its formula (not yet metadata)
* Neutralizing a variable: "removing" the variable from the calculation tree by having it return default values (usually 0)

In the case of our example reform, we only want to update the existing EITC variable, as so:

In [3]:
# policyengine_us/reforms/example_reforms/eitc_reform.py

from policyengine_core.reforms import StructuralReform
from policyengine_us.reforms.example_variables.eitc import eitc

eitc_reform = StructuralReform("gov.contrib.example_parameters.eitc_reform")
eitc_reform.update_variable(eitc)

# Also allowable are the below:
# example_reform.add_variable(eitc) (if it didn't already exist in the tax-benefit system)
# example_reform.neutralize_variable(eitc)

For this reform to run successfully, we'd need to ensure a "trigger parameter" exists at `gov.contrib.example_parameters.eitc_reform`. This is a Boolean parameter that is set to `False` by default, but that a user might set to `True`. It can be set back to `False` if a user is looking to activate a reform only temporarily, but cannot be set to `True` more than once in our current implementation. Here's what this parameter might look like:

```
# policyengine_us/parameters/gov/contrib/example_parameters/eitc_reform.yaml
description: Test param for test struct reform
values:
  2000-01-01: False
metadata:
  unit: bool
  label: Test trigger param for test struct reform
```

We will also need to add it to `policyengine_us/reforms/reforms.py` in much the same way that we do currently. This remains under construction, but for the time being looks like this:

In [4]:
# policyengine_us/reforms/reforms.py

from policyengine_us.reforms.example_reforms.eitc_reform import test_reform

structural_reforms = [
  test_reform
]

def create_structural_reforms_from_parameters(parameters, tax_benefit_system):
    for reform in structural_reforms:
        reform.activate(
            tax_benefit_system=tax_benefit_system,
        )

ImportError: cannot import name 'test_reform' from 'policyengine_us.reforms.example_reforms.eitc_reform' (/Users/administrator/Documents/PolicyEngine/policyengine-us/policyengine_us/reforms/example_reforms/eitc_reform.py)

I've gone ahead and rigged up this reform in the repo. If done successfully, the below cell can be run and should show the impacts of our reform when applied from 2027 onward, but calculated over 2025 and 2028.

In [2]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

reform = Reform.from_dict({
  "gov.contrib.example_parameters.eitc_reform": {
    "2027-01-01.2100-12-31": True
  }
}, country_id="us")


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)

# Calculate baseline EITC
baseline_eitc = baseline.calculate("eitc", period=2025)

# Calculate EITC under a reform that doesn't take effect 
# until 2028 -> should be the same as baseline
reformed_eitc_pre = reformed.calculate("eitc", period=2025)

# Calculate EITC under a reform that takes effect in 2028, where
# the reform makes the EITC "4" for everyone who qualifies for it
reformed_eitc_post = reformed.calculate("eitc", period=2028)

# These two should be the same value
print(baseline_eitc.sum())
print(reformed_eitc_pre.sum())

# This one should be quite different
print(reformed_eitc_post.sum())

# This should show that each tax unit that qualifies receives $4
print(reformed_eitc_post)

print("Baseline TBS struct reforms:")
print(baseline.tax_benefit_system.possible_structural_reforms)
print("Reformed TBS struct reforms:")
print(reformed.tax_benefit_system.possible_structural_reforms)

create_structural_reforms_from_parameters
reform.activate
create_structural_reforms_from_parameters
reform.activate
create_structural_reforms_from_parameters
reform.activate
create_structural_reforms_from_parameters
reform.activate
34862057602.782974
34862057602.782974
452409307.43963623
       value       weight
0        0.0   628.313721
1        0.0   628.313721
2        4.0   628.313721
3        4.0  1629.341675
4        0.0  1198.733154
...      ...          ...
77421    4.0   732.707092
77422    4.0   605.711609
77423    4.0   400.256622
77424    4.0   400.256622
77425    4.0   747.214600

[77426 rows x 2 columns]
Baseline TBS struct reforms:
Reformed TBS struct reforms:


In [3]:
for item in reformed.tax_benefit_system.possible_structural_reforms:
    print(item.transformation_log)
    for trans_log_item in item.transformation_log:
        print(trans_log_item.variable.formula)

[TransformationLogItem(variable_name='eitc', variable=<class 'policyengine_us.reforms.example_variables.eitc.eitc'>, transformation='update')]
<function eitc.formula at 0x1268e7d00>
